In [7]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-

import time

import matplotlib.pyplot as plt
import numpy as np


def round_up(num):
    """
    Returns the multiple of ten next to `num`.
    """
    if num % 10:
        return (10 - num % 10) + num
    else:
        return num

class Chromosome:
    """
    Implements basic structure for
    storing and operating on algorithm data.
    """
    def __init__(self, size=0, alphabet=[]):
        if not size:
            self.genes = np.ndarray(size)
        else:
            np.random.seed()
            self.genes = np.random.choice(alphabet, size)

        self.fitness = 0

    def __str__(self):
        return "".join(list(map(str, self.genes)))

    def calculate_fitness(self, target):
        """
        Calculates fitness of the individual
        given a target.
        """
        hits = 0
        for i, j in zip(self.genes, target):
            if i == j:
                hits += 1
        self.fitness = (hits * 100) / len(target)


class Population:
    """
    Implements basic operations on a set
    of Chromosome objects.
    """

    def __init__(self, alphabet, psize, csize):
        """
        :param alphabet: Symbols used to generate the chromosomes.
        :param psize: Population size.
        :param csize: Chromosome size.
        """
        self.alphabet = alphabet
        self.psize = psize
        self.csize = csize
        self.population = self._gen_pop(psize, csize, alphabet)

    def __str__(self):
        return "\n".join(map(str, self.population))

    def update(self, target):
        """
        Updates the fitness value for every
        individual in the population based on
        the given target.
        """
        for c in self.population:
            c.calculate_fitness(self.target)

    def get_fitness(self):
        """
        Returns the fitness value for every
        individual in the population.
        """
        return [c.fitness for c in self.population]

    def sort(self):
        """
        Sorts the elements of the population
        based on its fitness values.
        """
        self.population = sorted(self.population, key=lambda chromosome: chromosome.fitness)

    def renew(self):
        """
        Eliminates the half of the population
        with the worst fitness values, replacing
        them with a list of new randomly generated
        individuals.
        """
        index = int(len(self.population) / 2)
        self.population = self._gen_pop(index, self.csize, self.alphabet) \
                          + self.population[index:]

    def _gen_pop(self, psize, csize, alphabet):
        """
        Generates a new list of individuals randomly.

        :param psize: Size of the population to be generated.
        :param csize: Size of the Chromosome to be generated.
        :param alphabet: Set of symbols used to generate the population individuals.
        :return: A list of randomly generated individuals.
        """
        return [Chromosome(csize, alphabet) for _ in range(psize)]


class Randomwalk(Population):
    """
    Implements the steps of a Randomwalk
    algorithm based on the methods of the
    Population class.
    """
    def __init__(self, alphabet, target, psize, gcounter, tcounter=30):
        """

        :param alphabet: Symbols used to generate the population individuals.
        :param target: The target to be achieved by the algorithm.
        :param psize: Population size.
        :param gcounter: Max generations without change in best fitness permited.
        :param tcounter: Number of times the algorithm is executed.
        """
        self.alphabet = alphabet
        self.target = target
        self.csize = len(self.target)
        self.psize = psize
        self.gcounter = gcounter
        self.tcounter = tcounter

    def run(self, verbose=False):
        """
        Routine that combines the methods of the
        base class (Population) to perform the algorithm.

        :param verbose: If **True**, logs info during the execution of the algorithm.
        """

        # Initiate a list to receive the best fitness values
        # achieved per execution.
        self.best_scores = []

        # Initiate a list to receive the number of generations
        # spent per execution.
        self.generations = []

        if verbose:
            then = time.time()

            print("{!s} started...".format(type(self).__name__))
            print("Target: {!s}".format("".join(map(str, self.target))))
            print("Population size: {!s}".format(self.psize))
            print("Total executions: {!s}".format(self.tcounter))
            print("Max generations without change: {!s}\n".format(self.gcounter))

        # Runs the algorithm tcounter times.
        for i in range(self.tcounter):

            # Initiates the population and updates its fitness values
            # based on a given target.
            self.population = self._gen_pop(self.psize, self.csize, self.alphabet)
            self.update(self.target)

            # Initiates a list to receive the best fitness values
            # achieved per generation.
            scores = []

            j = 0  # Holds the value of the best fitness per generation.
            k = 1  # Holds the current generation.
            g = 0  # Holds the number of generations with no changes to the best fitness.

            if verbose:
                print("\nExecution {!s} started...\n".format(i + 1))

            while g < self.gcounter:

                self.sort()
                self.renew()
                self.update(self.target)
                self.sort()

                j = self.get_fitness()[-1]
                scores.append(j)

                # Only compares the last two elements of
                # score, if at least 2 generations had
                # been executed.
                if k >= 2:
                    if scores[-1] == scores[-2]:
                        g += 1
                    else:
                        g = 0

                if verbose:
                    print("\tGeneration {!s}".format(k))
                    print("\tBest individual: {!s}".format(self.population[-1]))
                    print("\tBest score: {!s}%\n".format(j))

                if verbose and j == 100:
                    print("\tTarget found!\n")
                    break

                k += 1

            self.best_scores.append(scores)
            self.generations.append(k - 1)

            if verbose:
                print("Execution {0!s} ended in {1!s} generations".format(i + 1, k - 1))
                print("Target: {!s}".format("".join(map(str, self.target))))
                print("Result: {!s}".format(self.population[-1]))
                print("Best score: {!s}%\n".format(scores[-1]))

        # Holds the number of the longest execution, rounded up to a multiple of ten.
        lexec = round_up(max([len(bscore) for bscore in self.best_scores]))

        # Normalizing the length of the scores lists,
        # repeating its last element until it fits lexec.
        for bscore in self.best_scores:
            bscore.extend([bscore[-1]] * (lexec - len(bscore)))

        # Calculating the mean and standard deviation
        # per generation.
        self.m = np.mean(self.best_scores, axis=0)
        self.std = np.std(self.best_scores, axis=0)

        # Creating a numpy array to be used index
        # by the plot method.
        self.idx = np.arange(1, lexec + 1)

        if verbose:
            now = time.time()

            # Sets a maximum of 4 decimal places for printing floats.
            np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

            print("Mean score per generation:")
            print(self.m)
            print()
            print("Standard deviantion per generation:")
            print(self.std)
            print()
            print("Generations spent per execution:")
            print(self.generations)
            print()
            print("{!s} ended in {!s} seconds.".format(type(self).__name__, now - then))

    def plot(self):
        """
        Plots a graph displaying the mean values and the standard
        deviation per generation of the fitness values achieved.
        """

        plt.figure(figsize=(15, 10))
        plt.title(u"Média de Acertos por Geração")
        plt.xlabel(u"Gerações")
        plt.ylabel(u"Acertos (%)")
        plt.grid(alpha=0.3)
        plt.errorbar(self.idx, self.m, self.std, ls=None, color='r', ecolor='y', alpha=0.5)
        plt.show()


class GeneticAlgorithm(Population):
    """
    Implements the steps of a Genetic
    Algorithm based on the methods of
    the base class (Population) and on
    its own methods.
    """

    def __init__(self, alphabet, target, psize, rsize, mrate, brate, sigma, gcounter, tcounter=30):
        """

        :param alphabet: Set of symbols used to generate the individuals of the population.
        :param target: The target to be achieved by the algorithm.
        :param psize: Population size.
        :param rsize: Ring size.
        :param mrate: Mutation rate.
        :param brate: Breeding rate.
        :param sigma: Standard deviation used to propagate the mutation over an individual genes.
        :param gcounter: Max number of generations to run with no change to the best fitness per execution.
        :param tcounter: Number of times the algorithm is executed.
        """
        self.alphabet = alphabet
        self.target = target
        self.psize = psize
        self.rsize = rsize
        self.csize = len(self.target)
        self.mrate = mrate
        self.brate = brate
        self.sigma = sigma
        self.gcounter = gcounter
        self.tcounter = tcounter

    def select(self):
        selected = []
        threshold = 1 - self.rsize/self.psize

        for _ in range(self.psize):
            contenders = sorted(np.random.choice(self.population, self.rsize),
                                key=lambda chromosome: chromosome.fitness)
            if np.random.rand() < threshold:
                selected.append(contenders[-1])
            else:
                selected.append(contenders[0])
        return selected

    def breed(self, parents, points=2):
        children = []

        for i in range(0, self.psize -1, 2):
            couple = [parents[i], parents[i+1]]
            if np.random.rand() > self.brate:
                children.extend(couple)
            else:
                ch1, ch2 = couple[0], couple[1]

                pts = np.random.randint(0, self.csize, size=points)

                while pts[0]-pts[1] <= 1:
                    pts = np.random.randint(0, self.csize, size=2)

                ch1.genes[pts[1]:pts[0]], ch2.genes[pts[0]:pts[1]] = couple[1].genes[pts[1]:pts[0]], couple[0].genes[pts[0]:pts[1]]

                children.extend([ch1, ch2])

        return children

    def mutate(self):
        for chromosome in self.population:
            for i in range(1, self.csize - 1):
                if np.random.rand() < self.mrate:
                    chromosome.genes[i] = int(not(chromosome.genes[i]))

                    s = np.random.normal(0, self.sigma)
                    if not (s > -1.5*self.sigma and 1.5*self.sigma > s):
                        chromosome.genes[i-1] = int(not(chromosome.genes[i-1]))
                        chromosome.genes[i+1] = int(not(chromosome.genes[i+1]))
            # chromosome.calculate_fitness(self.target)

    #def __elitefy(self, elite):
    #    length = self.psize/2
    #    idx = np.random.randint(0, length)
    #    
    #    self.population[idx] = elite

    def run(self, verbose=False):
        """
        Routine that combines the methods of the
        base class (Population) to perform the algorithm.

        :param verbose: If **True**, logs info during the execution of the algorithm.
        """

        # Initiate a list to receive the best fitness values
        # achieved per execution.
        self.best_scores = []

        # Initiate a list to receive the number of generations
        # spent per execution.
        self.generations = []

        if verbose:
            then = time.time()

            print("{!s} started...".format(type(self).__name__))
            print("Target: {!s}".format("".join(map(str, self.target))))
            print("Population size: {!s}".format(self.psize))
            print("Total executions: {!s}".format(self.tcounter))
            print("Max generations without change: {!s}\n".format(self.gcounter))

        # Runs the algorithm tcounter times.
        for i in range(self.tcounter):

            # Initiates the population and updates its fitness values
            # based on a given target.
            self.population = self._gen_pop(self.psize, self.csize, self.alphabet)
            self.update(self.target)
            self.sort()

            # Initiates a list to receive the best fitness values
            # achieved per generation.
            scores = []

            j = 0  # Holds the value of the best fitness per generation.
            k = 1  # Holds the current generation.
            g = 0  # Holds the number of generations with no changes to the best fitness.

            elite = self.population[-1]

            if verbose:
                print('\nExecution {!s} started...\n'.format(i + 1))

            while g < self.gcounter:
                
                #print("Gen {!s}\n".format(k))
                #print("Elite ({!s}): {!s}".format(id(elite), elite.fitness))
                #print("Fitness:\n{!s}\n".format([c.fitness for c in self.population]))
                
                selected = self.select()
                self.population = self.breed(selected)
                self.mutate()
                self.update(self.target)
                self.sort()
                
                """
                if self.population[-1].fitness > elite.fitness:
                    elite = self.population[-1]
                    print("New elite ({!s}): {!s}".format(id(elite), elite.fitness))
                    print("Fitness:\n{!s}\n".format([c.fitness for c in self.population]))
                    
                elif elite not in self.population:
                    length = self.psize/2
                    idx = np.random.randint(0, length)

                    self.population[idx] = elite
                    
                    print("Elite returned ({!s}): {!s}".format(id(elite), elite.fitness))
                    print("Fitness before sorting:\n{!s}".format([c.fitness for c in self.population]))
                    self.sort()
                    print("Fitness after sorting:\n{!s}\n".format([c.fitness for c in self.population]))
                    
                else: 
                    print("Elite ({!s} {!s}) not changed.".format(id(elite), elite.fitness))
                    print("Fitness:\n{!s}\n".format([c.fitness for c in self.population]))
                    
                print("End of Gen {!s}".format(k))
                print("Fitness:\n{!s}\n".format([c.fitness for c in self.population]))
                """

                j = self.get_fitness()[-1]
                scores.append(j)#elite.fitness)

                # Only compares the last two elements of
                # score, if at least 2 generations had
                # been executed.
                if k >= 2:
                    if scores[-1] == scores[-2]:
                        g += 1
                    else:
                        g = 0

                if verbose:
                    print("\tGeneration {!s}".format(k))
                    print("\tBest individual: {!s}".format(self.population[-1]))
                    print("\tBest score: {!s}%\n".format(self.population[-1].fitness))

                if elite.fitness == 100:
                    print("\tTarget found!\n")
                    break

                k += 1

            self.best_scores.append(scores)
            self.generations.append(k - 1)

            if verbose:
                print("Execution {0!s} ended in {1!s} generations".format(i + 1, k - 1))
                print("Target: {}".format(self.target))
                print("Result: {}".format(self.population[-1]))
                print("Best score: {!s}%\n".format(scores[-1]))

        # Holds the number of the longest execution, rounded up to a multiple of ten.
        lexec = round_up(max([len(bscore) for bscore in self.best_scores]))

        # Normalizing the length of the scores lists,
        # repeating its last element until it fits lexec.
        for bscore in self.best_scores:
            bscore.extend([bscore[-1]] * (lexec - len(bscore)))

        # Calculates the mean and standard deviation
        # per generation.
        self.m = np.mean(self.best_scores, axis=0)
        self.std = np.std(self.best_scores, axis=0)

        # Creating a numpy array to be used index
        # by the plot method.
        self.idx = np.arange(1, lexec + 1)

        if verbose:
            now = time.time()

            # Sets a maximum of 4 decimal places for printing floats.
            np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

            print("Mean score per generation:")
            print(self.m)
            print()
            print("Standard deviantion per generation:")
            print(self.std)
            print()
            print("Generations spent per execution:")
            print(self.generations)
            print()
            print("{!s} ended in {!s} seconds.".format(type(self).__name__, now - then))

    def plot(self):
        """
        Plots a graph displaying the mean values and the standard
        deviation per generation of the fitness values achieved.
        """

        plt.figure(figsize=(15, 10))
        plt.title(u"Média de Acertos por Geração")
        plt.xlabel(u"Gerações")
        plt.ylabel(u"Acertos (%)")
        plt.grid(alpha=0.3)
        plt.errorbar(self.idx, self.m, self.std, ls=None, color='b', ecolor='g', alpha=0.5)
        plt.show()


In [12]:
t1 = [1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1]
t2 = [1, 0, 1, 0, 0, 0, 0, 1, 1, 1]

psize1  =   20
psize2  =  100
psize3  =  500
psize4  = 1000
psize5  = 5000

gcount = 5

In [14]:
ag11 = GeneticAlgorithm(alphabet=[0, 1], target=t2, psize=psize1, rsize=5, mrate=0.01, brate=0.5, sigma=0.5, gcounter=gcount, tcounter=30)

In [13]:
ag11.run(verbose=True)

GeneticAlgorithm started...
Target: 1010000111
Population size: 20
Total executions: 30
Max generations without change: 10


Execution 1 started...

	Generation 1
	Best individual: 1011000100
	Best score: 70.0%

	Generation 2
	Best individual: 1010010011
	Best score: 80.0%

	Generation 3
	Best individual: 1011110111
	Best score: 70.0%

	Generation 4
	Best individual: 1111010101
	Best score: 60.0%

	Generation 5
	Best individual: 1111010101
	Best score: 60.0%

	Generation 6
	Best individual: 1110010101
	Best score: 70.0%

	Generation 7
	Best individual: 1111000111
	Best score: 80.0%

	Generation 8
	Best individual: 1111010111
	Best score: 70.0%

	Generation 9
	Best individual: 1111010111
	Best score: 70.0%

	Generation 10
	Best individual: 1111010111
	Best score: 70.0%

	Generation 11
	Best individual: 1111010111
	Best score: 70.0%

	Generation 12
	Best individual: 1111000111
	Best score: 80.0%

	Generation 13
	Best individual: 0111000110
	Best score: 60.0%

	Generation 14
	Best individ

	Generation 201
	Best individual: 1111000000
	Best score: 50.0%

	Generation 202
	Best individual: 1111000000
	Best score: 50.0%

	Generation 203
	Best individual: 1111011000
	Best score: 30.0%

	Generation 204
	Best individual: 1101011010
	Best score: 30.0%

	Generation 205
	Best individual: 1101011010
	Best score: 30.0%

	Generation 206
	Best individual: 1001011010
	Best score: 40.0%

	Generation 207
	Best individual: 1011001110
	Best score: 70.0%

	Generation 208
	Best individual: 1011001110
	Best score: 70.0%

	Generation 209
	Best individual: 1011100110
	Best score: 70.0%

	Generation 210
	Best individual: 1111100000
	Best score: 40.0%

	Generation 211
	Best individual: 1100011000
	Best score: 30.0%

	Generation 212
	Best individual: 1100011011
	Best score: 50.0%

	Generation 213
	Best individual: 1100011011
	Best score: 50.0%

	Generation 214
	Best individual: 1000011011
	Best score: 60.0%

	Generation 215
	Best individual: 1000111111
	Best score: 60.0%

	Generation 216
	Best ind

	Generation 400
	Best individual: 1101001111
	Best score: 60.0%

	Generation 401
	Best individual: 1101001111
	Best score: 60.0%

	Generation 402
	Best individual: 1101011111
	Best score: 50.0%

	Generation 403
	Best individual: 1001011111
	Best score: 60.0%

	Generation 404
	Best individual: 1001101011
	Best score: 50.0%

	Generation 405
	Best individual: 1001101011
	Best score: 50.0%

	Generation 406
	Best individual: 1001101011
	Best score: 50.0%

	Generation 407
	Best individual: 1001101011
	Best score: 50.0%

	Generation 408
	Best individual: 1001110111
	Best score: 60.0%

	Generation 409
	Best individual: 1001011111
	Best score: 60.0%

	Generation 410
	Best individual: 1101010111
	Best score: 60.0%

	Generation 411
	Best individual: 1111000011
	Best score: 70.0%

	Generation 412
	Best individual: 1010000011
	Best score: 90.0%

	Generation 413
	Best individual: 1111001011
	Best score: 60.0%

	Generation 414
	Best individual: 1111011011
	Best score: 50.0%

	Generation 415
	Best ind

	Generation 598
	Best individual: 1111111101
	Best score: 40.0%

	Generation 599
	Best individual: 1011111101
	Best score: 50.0%

	Generation 600
	Best individual: 1011111101
	Best score: 50.0%

	Generation 601
	Best individual: 1011110111
	Best score: 70.0%

	Generation 602
	Best individual: 1011110111
	Best score: 70.0%

	Generation 603
	Best individual: 1011110111
	Best score: 70.0%

	Generation 604
	Best individual: 1111100111
	Best score: 70.0%

	Generation 605
	Best individual: 1011100111
	Best score: 80.0%

	Generation 606
	Best individual: 1001100010
	Best score: 50.0%

	Generation 607
	Best individual: 1011100010
	Best score: 60.0%

	Generation 608
	Best individual: 1011000010
	Best score: 70.0%

	Generation 609
	Best individual: 1110100110
	Best score: 70.0%

	Generation 610
	Best individual: 1000111010
	Best score: 40.0%

	Generation 611
	Best individual: 1111111010
	Best score: 30.0%

	Generation 612
	Best individual: 1101110010
	Best score: 30.0%

	Generation 613
	Best ind

	Generation 791
	Best individual: 1010010110
	Best score: 80.0%

	Generation 792
	Best individual: 1010000110
	Best score: 90.0%

	Generation 793
	Best individual: 1110100110
	Best score: 70.0%

	Generation 794
	Best individual: 1101000110
	Best score: 60.0%

	Generation 795
	Best individual: 1101001110
	Best score: 50.0%

	Generation 796
	Best individual: 1101001110
	Best score: 50.0%

	Generation 797
	Best individual: 1100100100
	Best score: 50.0%

	Generation 798
	Best individual: 0010100100
	Best score: 60.0%

	Generation 799
	Best individual: 0110000000
	Best score: 50.0%

	Generation 800
	Best individual: 0110010000
	Best score: 40.0%

	Generation 801
	Best individual: 0010100010
	Best score: 60.0%

	Generation 802
	Best individual: 0010100010
	Best score: 60.0%

	Generation 803
	Best individual: 0010100010
	Best score: 60.0%

	Generation 804
	Best individual: 0010000110
	Best score: 80.0%

	Generation 805
	Best individual: 0010000010
	Best score: 70.0%

	Generation 806
	Best ind

...

	Generation 642
	Best individual: 0001110100
	Best score: 30.0%

	Generation 643
	Best individual: 0001110100
	Best score: 30.0%

	Generation 644
	Best individual: 0001001100
	Best score: 40.0%

	Generation 645
	Best individual: 0011001100
	Best score: 50.0%

	Generation 646
	Best individual: 0011001100
	Best score: 50.0%

	Generation 647
	Best individual: 0011001100
	Best score: 50.0%

	Generation 648
	Best individual: 0110011100
	Best score: 40.0%

	Generation 649
	Best individual: 0110001000
	Best score: 40.0%

	Generation 650
	Best individual: 0110001100
	Best score: 50.0%

	Generation 651
	Best individual: 0101011100
	Best score: 20.0%

	Generation 652
	Best individual: 0101011100
	Best score: 20.0%

	Generation 653
	Best individual: 0111011100
	Best score: 30.0%

	Generation 654
	Best individual: 0111011110
	Best score: 40.0%

	Generation 655
	Best individual: 0111001110
	Best score: 50.0%

	Generation 656
	Best individual: 0111001110
	Best score: 50.0%

	Generation 657
	Best ind

	Generation 839
	Best individual: 0001001110
	Best score: 50.0%

	Generation 840
	Best individual: 0000011100
	Best score: 40.0%

	Generation 841
	Best individual: 0000011100
	Best score: 40.0%

	Generation 842
	Best individual: 0000010100
	Best score: 50.0%

	Generation 843
	Best individual: 0000000100
	Best score: 60.0%

	Generation 844
	Best individual: 0000010000
	Best score: 40.0%

	Generation 845
	Best individual: 0001010100
	Best score: 40.0%

	Generation 846
	Best individual: 0001011000
	Best score: 20.0%

	Generation 847
	Best individual: 0000111010
	Best score: 30.0%

	Generation 848
	Best individual: 0101111010
	Best score: 10.0%

	Generation 849
	Best individual: 0101011010
	Best score: 20.0%

	Generation 850
	Best individual: 0111000010
	Best score: 50.0%

	Generation 851
	Best individual: 0111000010
	Best score: 50.0%

	Generation 852
	Best individual: 0111000010
	Best score: 50.0%

	Generation 853
	Best individual: 0111000010
	Best score: 50.0%

	Generation 854
	Best ind

	Generation 1034
	Best individual: 0011010111
	Best score: 70.0%

	Generation 1035
	Best individual: 0011010111
	Best score: 70.0%

	Generation 1036
	Best individual: 0011010011
	Best score: 60.0%

	Generation 1037
	Best individual: 0011010011
	Best score: 60.0%

	Generation 1038
	Best individual: 0011000011
	Best score: 70.0%

	Generation 1039
	Best individual: 0011010011
	Best score: 60.0%

	Generation 1040
	Best individual: 0011000011
	Best score: 70.0%

	Generation 1041
	Best individual: 0111100001
	Best score: 40.0%

	Generation 1042
	Best individual: 0110100001
	Best score: 50.0%

	Generation 1043
	Best individual: 0111100001
	Best score: 40.0%

	Generation 1044
	Best individual: 0101100101
	Best score: 40.0%

	Generation 1045
	Best individual: 0101100101
	Best score: 40.0%

	Generation 1046
	Best individual: 0101100101
	Best score: 40.0%

	Generation 1047
	Best individual: 0100101001
	Best score: 30.0%

	Generation 1048
	Best individual: 0100101110
	Best score: 40.0%

	Generatio

	Generation 1235
	Best individual: 0001100101
	Best score: 50.0%

	Generation 1236
	Best individual: 0001100101
	Best score: 50.0%

	Generation 1237
	Best individual: 0101100010
	Best score: 30.0%

	Generation 1238
	Best individual: 0111100010
	Best score: 40.0%

	Generation 1239
	Best individual: 0101100010
	Best score: 30.0%

	Generation 1240
	Best individual: 0100010110
	Best score: 50.0%

	Generation 1241
	Best individual: 0100010010
	Best score: 40.0%

	Generation 1242
	Best individual: 0100010000
	Best score: 30.0%

	Generation 1243
	Best individual: 0100010000
	Best score: 30.0%

	Generation 1244
	Best individual: 0100010000
	Best score: 30.0%

	Generation 1245
	Best individual: 0101110000
	Best score: 10.0%

	Generation 1246
	Best individual: 1011110100
	Best score: 50.0%

	Generation 1247
	Best individual: 1010010100
	Best score: 70.0%

	Generation 1248
	Best individual: 1010010100
	Best score: 70.0%

	Generation 1249
	Best individual: 1110011100
	Best score: 50.0%

	Generatio

KeyboardInterrupt: 

In [ ]:
rw11 = Randomwalk(alphabet=[0, 1], target=t1, psize=psize1, gcounter=gcount, tcounter=30)

In [34]:
a = [Chromosome() for _ in range(10)]

In [35]:
a

In [36]:
b = a[-1]


In [37]:
a[-1] = Chromosome()

In [39]:
a

In [40]:
b